Here we implement the solver from [Quantum algorithm for solving linear differential equations: Theory and experiment](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307) by Xin *et al.* in the **Classiq** quantum programming language. 

### Classical Problem

Xin *et al.* provide a new method to solve any Linear Differential Equation formulated as 

$$\frac{d}{dt}\vec{x}(t)= M\vec{x}(t)+\vec{b}$$

for an $N\times N$ matrix $M$ and $N$-dimensional vectors $x$, $b$. 

Taylor-expanding the analytical solution, 

$$
\begin{aligned}
x(t)&=e^{Mt}x(0)+\left(e^{Mt}-\mathbb{I}\right)M^{-1}b
\\
&\approx \sum^k_{m=0}\frac{(Mt)^m}{m!}x(0)+\sum^k_{n=1}\frac{M^{n-1}t^n}{n!}b.
\end{aligned}
$$

### Quantum Formulation

1. Prepare two quantum states $\ket{x(0)}$ and $\ket{b}$ in the $N$-dimensional computational basis, on $\log_2(N)$ qubits. 
1. Describe the $N\times N$ operator $M$ as $A$ in the computational basis, such that $A_{ij}=M_{ij}\ket{i}\bra{j}$ up to normalization.

We then arrive at 

$$\ket{x(t)}\approx \sum_{m=0}^k \frac{\|x(0)\|(\|M\|At)^m}{m!}\ket{x(0)}+\sum_{n=1}^k\frac{\|b\|(\|M\|A)^{n-1}t^n}{n!}\ket{b}$$


#### More Encodings
1. Assuming $A$ is unitary, encode $k$ powers of $A$ as the unitaries $U_n=A^n$.
1. Define $C_m=\|x(0)\|(\|M\|t)^m/m!$ and $D_n=\|b\|(\|M\|t)^{n-1}t/n!$



Import all relevant Classiq modules and functions! You need to install `classiq` from `pip` beforehand. You can do this with Anaconda via `conda env create -f environment.yml`. 

In [4]:
import numpy as np
import scipy.linalg as la
import math
from util import make_quantum, is_unitary
from Code_For_V_and_W_Operators import V_operator, normalization, construct_VS1_VS2, construct_W_WS1_WS2
import time

from classiq import (
    qfunc, Output, QBit, QNum, QArray, CInt,
    allocate, within_apply, control, repeat, if_, 
    inplace_prepare_state, hadamard_transform, inplace_prepare_int, 
    inplace_prepare_amplitudes, show, 
    create_model, write_qmod,
    X, H, SWAP, unitary,
    Constraints, Preferences, 
)
from classiq.synthesis import synthesize, set_constraints, set_execution_preferences, set_preferences
from classiq.executor import execute
from classiq.execution import ExecutionPreferences
from classiq.interface.generator.quantum_program import QuantumProgram

To submit the code to Classiq and process it, we wrap all of the native `qfunc`s with a larger function that allows us to vary the order of the Taylor approximation $k$, the time $t$, and the error bound of the amplitude encoding.

In [ ]:
def write_qprog(qprog, fname):
    '''
    A simple function that saves your code as a text file
    '''
    file = open(fname,"w")
    file.write(qprog)
    file.close()

def create_LDE_qmod(x0, b, U, Cvals, Dvals, 
                    k=2, err_bound=0.01,
                    save_qmod='qmod/test'):
    """
    Solve LDE using thingss... 
    U: list of k powers of A
    """
    nqubits = math.ceil(np.log2(len(x0)))
    
    def inp_multiple_registers(*args):
        """ prepare multiple registers at once as a single function
        Inputs: register1, values1, register2, values2, ...

        - useful for doing many things in a single classiq control block
        """
        for i in range(0, len(args), 2):
            inplace_prepare_amplitudes(amplitudes=args[i+1], 
                                       out=args[i], bound=err_bound)

    @qfunc
    def prepare_registers(x0_b_ancilla: QBit, work_register: QArray, taylor_register: QNum):
        # evolve ancilla into superposition state
        inp_multiple_registers(x0_b_ancilla, [C/N,D/N])
        # encode x_0 and Cvals into registers with |0> ancilla
        control(x0_b_ancilla == 0, 
                lambda: inp_multiple_registers(work_register, x0, taylor_register, Cvals))
        # encode b and Dvals into registers with |1> ancilla
        control(x0_b_ancilla == 1, 
                lambda: inp_multiple_registers(work_register, b, taylor_register, Dvals))

    @qfunc
    def do_entangling(taylor_register: QNum, work_register: QArray):
        # apply powers of A to taylor register
        repeat(count = k + 1,
            iteration=lambda i: control(taylor_register == i, 
                                        lambda: unitary(U[i], work_register))
        )

    @qfunc
    def main(x0_b_ancilla: Output[QBit], work_register: Output[QArray], 
            taylor_register: Output[QNum]):
        # allocate all qubits before doing anything
        allocate(1, x0_b_ancilla)
        allocate(nqubits, work_register)
        allocate(k+1, taylor_register)
        # apply V^dag U V
        within_apply(compute=lambda: prepare_registers(x0_b_ancilla, work_register, taylor_register),
                    action=lambda: do_entangling(taylor_register, work_register))
    
    # create the model!
    qmod = create_model(main)
    # store the .qmod code!
    write_qmod(qmod, save_qmod)
    print(type(qmod))
    return qmod

def run_qmod(qmod, opt='depth', nshots=10000, job_name='',
             save_qprog='qprog',
             show_circuit=True, print_circuit_info=True):
    """ optimize qmod with constraints and run it on a simulator """
    qmod = set_constraints(qmod,
        Constraints(max_width=25, optimization_parameter=opt))
    qmod = set_execution_preferences(qmod, 
        ExecutionPreferences(num_shots=nshots, job_name=job_name, 
                             random_seed='767'))
    qmod = set_preferences(qmod, 
        Preferences(backend_service_provider="Classiq", 
                    backend_name="simulator", 
                    timeout_seconds=600, optimization_timeout_seconds=120))
    start_time = time.time()
    qprog = synthesize(qmod)
    circuit_width = QuantumProgram.from_qprog(qprog).data.width
    circuit_depth = QuantumProgram.from_qprog(qprog).transpiled_circuit.depth
    end_time = time.time()
    if print_circuit_info:
        print(f"\ttook {end_time-start_time:.2f}s: width={circuit_width},depth={circuit_depth}")
    # open in viewer
    if show_circuit: 
        show(qprog)
    # save generated quantum program
    if save_qprog:
        write_qprog(qprog, save_qprog)
    start_time = time.time()
    
    job = execute(qprog)
    print(
        f"\tjob with {job.num_shots} shots is {job.status} on provider-backend={job.provider}-{job.backend_name} \n\tand can be accessed at {job.ide_url}"
    )
    results = job.result()[0].value
    end_time = time.time()
    print(f"\tjob took {end_time-start_time:.2f}s")
    return results.parsed_states

In [ ]:
def solve_all_times_ODE(x0,M,b,times, k=2,err_bound=0.01):
    for t in times:
        nbits = len(x0)
        A = M / np.linalg.norm(M, ord='fro')
        # store pre-conmputed powers of A
        U = [np.linalg.matrix_power(A, i) for i in range(0,k+1)]
        # store pre-computed V and W operators
        C, D, Cvals, Dvals = normalization(A, k, x0, b, t)
        N = np.sqrt(C**2 + D**2)

        solve_unitary_LDE()


